# CPIT 440 lab manual - Lab 5  & Lab 6
 
   ## Objectives
   
   These labs aim to practice the preprocessing steps to prepare the data to build the model. These steps will be as follwoing:  
   1. Splitting into train and test sets
   2. Divide the dataset into Dependent & Independent variable
   3. Data cleaning (missing values)
   4. Handling the categorical attributes
   5. Handling outliers
   6. Data transformation and scaling  
   

-------------------------------------------

In [ ]:
import pandas as pd
#url="https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv"
url="C:/Users/Asus/Desktop/IT/CPIT440/My Lab/My scripts/Lab 4 and 5 and 6/housing.csv"
housing=pd.read_csv(url)

In [ ]:
housing.info()

Note: you can find the book and all its materials (dataset and the notebooks) in this link: https://github.com/ageron/handson-ml

**Duplication:**

Sometimes we find duplicates in our dataset which cause problems in the analysis. We need to remove the redundant rows.  
  
To print the duplicated rows in the dataset (this code only checks for duplicates but does not remove them):

In [ ]:
housing.duplicated(subset=None, keep='first')

In [ ]:
dup_rows = housing[housing.duplicated(subset=None, keep='first')]
dup_rows

Based on the above result, housing dataset has no duplicates.  
  
The following code can be used to find and drop all the duplicated rows in any dataset.

```python
housing.drop_duplicates(subset=None, keep='first',inplace=True)
```

# 1. Sampling and Splitting into train and test sets

The train-test split is a technique for evaluating the performance of a machine learning algorithm. It can be used for classification or regression problems and can be used for any supervised learning algorithm.  
The procedure involves taking a dataset and dividing it into two subsets. The first subset is used to fit the model and is referred to as the training dataset. The second subset is not used to train the model; instead, the input element of the dataset is provided to the model, then predictions are made and compared to the expected values. This second dataset is referred to as the test dataset.  
The objective is to estimate the performance of the machine learning model on new data: data not used to train the model.  
The train-test procedure is appropriate when there is a sufficiently large dataset available that has suitable representation of the problem. A suitable representation of the problem domain means that there are enough records to cover all common cases and most uncommon cases in the domain. If you have insufficient data, then a suitable alternate model evaluation procedure would be the k-fold cross-validation procedure.   
  
Creating a test set is theoretically simple: just pick some instances **randomly**, typically 20% of the dataset, and set them aside. This is done to ensure that datasets are a representative sample (e.g. random sample) of the original dataset. We will need to set a variable called `random_state` to an integer value. Why? When comparing machine learning algorithms, it is desirable (perhaps required) that they are fit and evaluated on the same subsets of the dataset. This can be achieved by fixing the seed for the pseudo-random number generator used when splitting the dataset. This can be achieved by setting the `random_state` to an integer value. Any value will do; it is not a tunable hyperparameter. Moreover, when we repeat the run it is important that our random numbers generator, generates the same numbers for the test set to prevent our model from seeing all the dataset. This problem can be solved using the `random_state`.



In [ ]:
from sklearn.model_selection import train_test_split

train_set1, test_set1 = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
train_set1.shape

In [ ]:
test_set1.shape

In [ ]:
housing.shape

In the above method we used the **random sampling**. This is generally fine if your dataset is large enough (especially relative to the number of attributes), but if it is not, you run the risk of introducing a significant sampling bias. The test set should be a representative of the whole population. Sometimes, our data is skewed and we need to use **Stratified sampling** to take the same proportion from each part of the dataset.

The method `train_test_split` can be used to stratify by categorical variable. 

Suppose you chatted with experts who told you that the `median_income` is a very important attribute to predict median housing prices. You may want to ensure that the test set is representative of the various categories of incomes in the whole dataset. Since the median income is a continuous numerical attribute, you first need to create an income category attribute. Looking at the median income histogram more closely, most median income values are clustered around 2 to 5 (20,000–50,000) It is important to have a sufficient number of instances in your dataset for each stratum.




In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(column='median_income', bins=10)
plt.show()

The following code creates an income category attribute by multiplying the median income by (2/3) (to limit the number of income categories), and rounding up using ceil (to have discrete categories), and then keeping only the categories lower than 5 and merging the other categories into category 5:

In [ ]:
import numpy as np
housing["income_cat"] = np.ceil(housing["median_income"]*(2/3))
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)
housing["income_cat"].hist()
plt.show()

Now we are ready to apply the stratification.

In [ ]:
train_set2, test_set2 = train_test_split(housing, test_size=0.2, random_state=42,stratify=housing['income_cat'])

In [ ]:
housing["income_cat"].value_counts()/len(housing)

In [ ]:
train_set2["income_cat"].value_counts()/len(train_set2)

In [ ]:
test_set2["income_cat"].value_counts()/len(test_set2)

Now you should remove the income_cat attribute so the data is back to its original state:

In [ ]:
train_set2=train_set2.drop("income_cat", axis=1)
test_set2=test_set2.drop("income_cat", axis=1)


Note in the following cell that the index of the rows is shuffled during the sampling process. 

In [ ]:
train_set2

We can reset the index to start from 0.

In [ ]:
train_set2.reset_index(drop=True, inplace=True)
test_set2.reset_index(drop=True, inplace=True)

-------------------

# 2. Divide the dataset into Dependent & Independent variable

The next step would be to identify the independent variable (X or predictors) and the dependent variable (y, labels, or target). Assume that we want predict the house prices.

In [ ]:
train_X = train_set2.drop("median_house_value", axis=1)
train_y = train_set2["median_house_value"].copy()
test_X = test_set2.drop("median_house_value", axis=1)
test_y = test_set2["median_house_value"].copy()


--------------------

# 3. Data cleaning

The operations that are implemented in cleaning section and all the follwoing sections (section 4, 5, and 6) should be applied on both the train and the test sets. These steps are implemented after the spliting because we dont want to have any leakage from the test set to the train set when we calculate some central tendency measures. Data leakage happens when we create our model using information from outside the training dataset. This can produce optimistic and overfitted model that can fail to predict the unseen samples. Hence, the first step in the preprocessing should be splitting the data into train and test sets.   
To read more about data leakage: https://machinelearningmastery.com/data-leakage-machine-learning/ 
  
In the following cells I will apply the cleaning on the train set. The same steps should be repeated on the test set.  


In [ ]:
housing=train_X

### Missing values

Most Machine Learning algorithms cannot work with missing features, so let’s create a few functions to take care of them. You noticed earlier that the total_bedrooms attribute has some missing values, so let’s fix this.  
You have three options:  
* option 1: Get rid of the corresponding districts.
* option 2: Get rid of the whole attribute.
* option 3: Set the values to some value (zero, the mean, the median, etc.).  


First we will check for the null values: 

In [ ]:
#The following two commnads show the missing values
housing.info()
housing.isnull().sum(axis=0)  # sum of rows

To handle the missing values:

```python
# option 1
housing.dropna(subset=["total_bedrooms"],inplace=True) 
# option 2
housing.drop("total_bedrooms", axis=1,inplace=True) 
# option 3
median = housing["total_bedrooms"].median() 
housing["total_bedrooms"].fillna(median, inplace=True)
```

**Practice**  
The median cannot be calculated for the categorical attributes. If we have missing values in the attribute `ocean_proximity`, what is the suitable central tendency measure to use? Can you change the code in `# option 3` so it can handle categorical attribute? 

An alternative method to fill the missing values is to use a class in Scikit-Learn to take care of missing values: `SimpleImputer`. First, you need to create a `SimpleImputer` instance, specifying that you want to replace each attribute’s missing values with the median of that attribute. 

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np
imputer = SimpleImputer(missing_values=np.nan,strategy='median')


Since the median can only be computed on numerical attributes, we need to create a copy of the data without categorical attribute `ocean_proximity`.

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)
housing_num

Now you can fit the imputer instance to the data using the `fit()` method. In this method, the imputer computes the median of each attribute and stores the result in its `statistics_` instance variable.

In [ ]:
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
housing_num.median().values

Now you can use this trained imputer to transform the data set by replacing missing values by the learned medians.

In [ ]:
X = imputer.transform(housing_num)

The result is a NumPy array containing the transformed features. If you want to put it back into a Pandas DataFrame, it’s simple:

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns)

In [ ]:
housing_tr.info()

We finished the cleaning of the train set, but we droped the atrribute `ocean_proximity`. In the following section we will encode this categorical attribute before we restore it to our train set.

----------------

# 4 - Handeling the categorical attributes

In our data we have one categorical attribute `ocean_proximity`. Most Machine Learning algorithms prefer to work with numbers, since the models are based on mathematical equations and calculations. So let’s convert these categories from text to numbers. For this, we can use Pandas’ `factorize()` method which maps each category to a different integer.  
  
**Note:** The method explained in this section is suitable to categorical attributes in the input space `X`. If you work in classification task and want to encode the output `y`, you can use the class `LabelEncoder`. Moreover, the method is suitable for the nominal attributes. If you have an ordinal attribute in the input `X`, you can use the class `OrdinalEncoder`.

In [ ]:
housing_cat = housing["ocean_proximity"]
housing_cat_encoded, housing_categories = housing_cat.factorize()
print('housing_categories: ',housing_categories)
print('housing_cat_encoded: ',housing_cat_encoded)
print('housing_cat_encoded shape: ',housing_cat_encoded.shape)
type(housing_cat_encoded)   #it is a vector (one-dimensional structure)

One issue with this representation is that ML algorithms will assume that two nearby
values are more similar than two distant values. Obviously this is not the case. To fix this issue, a common solution is to create one binary attribute per category. I.e. convert the nominal attribute to many asymmetric binary attributes.  This is called one-hot encoding.  
  
  
**Number of columns = states of the categorical attribute**

Scikit-Learn provides a `OneHotEncoder` encoder to convert integer categorical values
into one-hot vectors.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1))
housing_cat_1hot

Note that `fit_transform()` expects a 2D array, but `housing_cat_encoded` is a 1D array, so we need to reshape it. Also, notice that the output is a SciPy sparse matrix, instead of a NumPy array. This is very useful when you have categorical attributes with thousands of categories. After one-hot encoding we get a matrix with thousands of columns, and the matrix is full of zeros except for a single 1 per row. Using up tons of memory mostly to store zeros would be very wasteful, so instead a sparse matrix only stores the location of the nonzero elements. You can use it mostly like a normal 2D array, but if you really want to convert it to a (dense) NumPy array, just call the `toarray()` method.

In [ ]:
housing_cat_1hot=housing_cat_1hot.toarray()

In [ ]:
print(type(housing_cat_1hot))
print(housing_cat_1hot[0:20,:])

Now we will convert `housing_cat_1hot` to a dataframe.

In [ ]:
housing_cat =pd.DataFrame(housing_cat_1hot,columns=housing_categories.tolist()) #contains categorical attributes
housing_cat

By the end of section 3 we got `housing_tr` which has numerical attributes without missing values. Also, by the end of section 4 we got `housing_cat` which has the encoded categorical attributes.  
We should concatenate them, however, first we need to check outliers and do scaling which will be done on the numerical attributes before concatenation.

--------------

# 5 - Handlig outliers

In statistics, an outlier is an observation point that is distant from other observations. The outliers can be a result of a mistake during data collection or it can be just an indication of variance in your data. Outliers can skew statistical measures and data distributions which produce bad prediction and analysis of the data.   
There are many methods to detect outliers in the dataset. Some of them are simple statistical methods such as methods that use standard deviations or the interquartile range. 

In the following we will apply the IQR (interquartile range) method.  
First, we will calculate the IQR of all the __numerical attributes__:

In [ ]:
Q1 = housing_tr.quantile(0.25)
Q3 = housing_tr.quantile(0.75)
IQR = Q3 - Q1
IQR

In [ ]:
outlier = ( housing_tr.min() < Q1-1.5*IQR) | (housing_tr.max() > Q3+1.5*IQR)
outlier

The above cell shows that 5 attributes have outliers.  
  
If we decided to remove all the rows that include outliers, we can use the code in the following cells:  


In the following code, we make boolean series such that the row that has outlier will be True. `DataFrame.any()` returns whether any element is True over requested axis.  


In [ ]:
#all the rows that has outliers will be True
outlier_index = ((housing_tr < (Q1 - 1.5 * IQR)) |(housing_tr > (Q3 + 1.5 * IQR))).any(axis=1)

outlier_index

To print the number of rows that include outlier in any column:

In [ ]:
print(outlier_index.value_counts())

In the following cell, we will print only the outlier rows:

In [ ]:
housing_tr[outlier_index]

The follwoing code will print the index of the outlier rows.

In [ ]:
index_numbers = housing_tr[outlier_index].index
index_numbers

Now we will drop all the rows that have outlier values.

In [ ]:
#Either using the method drop as follwoing
housing_out=housing_tr.drop(index_numbers)

# or using the following code
#housing_out = housing_tr[~outlier_index]
housing_out

A final thing we should mention here is that the above method is very simple and not adequate when we have high-dimensional input feature space. We will keep these outliers in our dataset and see the accuracy of the models. If you need more robust analysis of outliers, you can look at the following resources:  
1. Jiawei Han, Micheline Kamber and Jian Pei, "Data Mining Concepts and Techniques", Morgan Kaufmann Publishers, Third Edition, 2012. __Chapter 12: Outlier detection__
2. https://machinelearningmastery.com/model-based-outlier-detection-and-removal-in-python/. This site explains how to use `scikit-learn` library to find outliers using various robust methods.

--------------

# 6. Data transformation and scaling 

One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don’t perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. **Note that scaling the target values is generally not required**.

There are two common ways to get all attributes to have the same scale: min-max scaling and standardization (z-score normalization). Unlike min-max scaling, standardization does not bound values to a specific range, which may be a problem for some algorithms (e.g., neural networks often expect an input value ranging from 0 to 1). However, standardization is much less affected by outliers.  
Scikit-Learn provides a transformer called `StandardScaler` for standardization.  
It is important to fit the scalers to the training data only, not to the full dataset (including the test set). Only then can you use them to transform the training set and the test set (and new data). We will apply the scaling on the numerical features which are stored in `housing_tr` produced in section 3.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(housing_tr)
scaled=scaler.transform(housing_tr)
scaled

The result is a NumPy array containing the transformed features. If you want to put it back into a Pandas DataFrame, it’s simple:

In [ ]:
housing_scaled = pd.DataFrame(scaled, columns=housing_tr.columns)

In [ ]:
housing_scaled

Now we will merge the two dataframes (housing_scaled and housing_cat) in one dataframe to have all the numerical and categorical attributes in one dataset.

In [ ]:
housing_train = pd.concat([housing_scaled, housing_cat], axis=1)
housing_train
#housing.info()

Then we will export the train dataset to `csv` file.

In [ ]:
housing_train.to_csv('train_X.csv',index=False)
train_y.to_csv('train_y.csv',index=False)

In the follwoing cell I will repeat the steps to be implemented on the test set. Note that I will take the imputer, encoder, and scaler as fitted from the train data and use them to transform the test set to prevent data leakage.

In [ ]:
#missing values
housing = test_X
housing_num = housing.drop("ocean_proximity", axis=1)
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
#categorical attribute
housing_cat = housing["ocean_proximity"]
housing_cat_encoded, housing_categories = housing_cat.factorize()
housing_cat_1hot = encoder.transform(housing_cat_encoded.reshape(-1,1))
housing_cat_1hot=housing_cat_1hot.toarray()  #from sparse to dense
housing_cat =pd.DataFrame(housing_cat_1hot,columns=housing_categories.tolist()) 
#standardization
scaled=scaler.transform(housing_tr)
housing_scaled = pd.DataFrame(scaled, columns=housing_tr.columns)
#concatenation of numerical and categorical
housing_test = pd.concat([housing_scaled, housing_cat], axis=1)
#export to CSV files
housing_test.to_csv('test_X.csv',index=False)
test_y.to_csv('test_y.csv',index=False)

In [ ]:
housing_test

### References:
1. “Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems”, Aurélien Géron,O'Reilly Media; 2 edition (August 4, 2019)  
2. https://machinelearningmastery.com/model-based-outlier-detection-and-removal-in-python/
3. https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba